SEGUNDA PARTE: ANÁLISIS ESTADÍSTICO DE SEÑALES
Sara Palacio Duque 
CC:1007940687 

Comparación estadística de energia de señales de EGG

In [ ]:
#ejemplo archivo analisisEEG
import scipy.io as sio
import numpy as np
import os


directorio_actual=os.getcwd();
print(directorio_actual)

#listando los archivos en el directorio
archivos=os.listdir(directorio_actual + "/control")
print(len(archivos))

fs = 1000;
num_canales = 8;

#CARGAMOS LOS CONTROLES
for i in range(0,len(archivos)):
    #ruta de un archivo
    sujetoi=(directorio_actual + "/control" + "/" + archivos[i]);
    #1 CARGAR EL ARCHIVO
    data = sio.loadmat(sujetoi);
    data = data["data"];

    #2 VOLVERLO CONTINUO
    sensores = data.shape[0];
    puntos = data.shape[1];
    epocas = data.shape[2];

    senal_continua = np.reshape(data,(sensores, puntos*epocas), order = 'F');

    #APLICAR EL ANALISIS canal por canal
    #for c in range(0,num_canales):
        ####
        ####
        ####

#listando los archivos en el directorio
archivos2=os.listdir(directorio_actual + "/parkinson")
print(len(archivos2))


#CARGAMOS LOS PARKINSON
for i in range(0,len(archivos2)):
    #ruta de un archivo
    sujetoi=(directorio_actual + "/parkinson" + "/" + archivos2[i]);
    #1 CARGAR EL ARCHIVO
    data = sio.loadmat(sujetoi);
    data = data["data"];

    #2 VOLVERLO CONTINUO
    sensores = data.shape[0];
    puntos = data.shape[1];
    epocas = data.shape[2];

    senal_continua = np.reshape(data,(sensores, puntos*epocas), order = 'F');

    #APLICAR EL ANALISIS canal por canal
    #for c in range(0,num_canales):
        ###
        ###
        ###


c:\Users\sarap\Downloads\practica 3
36
23


1. Implemente una función que reciba una señal de múltiples canales y épocas y calcule la Energía de promedio de cada canal.

La energía de un canal se define como el promedio de x^2 en el tiempo, y luego se promedia sobre todas las épocas.

In [2]:
import numpy as np

def energy_per_channel(data):
    epochs_array = np.transpose(data, (2, 0, 1))   # (epocas, canales, puntos)
    energy_epochs = np.mean(epochs_array**2, axis=2)  
    energy_channels = np.mean(energy_epochs, axis=0)  
    return energy_channels


2. Calcule la energía de cada canal promediada por épocas para cada sujeto, esto para ambos grupos poblacionales. Guarde esta información en un DataFrame.

In [5]:
import os
import numpy as np
import pandas as pd
import scipy.io as sio


#fs = 1000
#num_canales = 8

# Construcción de DataFrame por grupo

def build_group_dataframe(folder):
    archivos = sorted(os.listdir(folder))
   
    rows = []
    subjects = []
    for fname in archivos:
        if not fname.endswith(".mat"):
            continue
        path = os.path.join(folder, fname)

        # Cargar archivo
        mat = sio.loadmat(path)
        data = mat["data"]  # (canales, puntos, epocas)

        # Calcular energía
        e_chan = energy_per_channel(data)

        rows.append(e_chan)
        subjects.append(fname.replace(".mat",""))

    # Crear DataFrame
    n_ch = len(rows[0])
    cols = [f"ch{c}" for c in range(n_ch)]
    df = pd.DataFrame(rows, index=subjects, columns=cols)

    # Guardar CSV en la misma carpeta
    out_path = os.path.join(folder, "energy.csv")
    df.to_csv(out_path)
    print(f"DataFrame de {folder} guardado en:", out_path)

    return df

# --------------------------
# Procesar ambos grupos
# --------------------------
df_control = build_group_dataframe(r"C:\Users\sarap\Downloads\practica 3\control")
df_parkinson = build_group_dataframe(r"C:\Users\sarap\Downloads\practica 3\parkinson")

print(df_control.head())
print(df_parkinson.head())



DataFrame de C:\Users\sarap\Downloads\practica 3\control guardado en: C:\Users\sarap\Downloads\practica 3\control\energy.csv
DataFrame de C:\Users\sarap\Downloads\practica 3\parkinson guardado en: C:\Users\sarap\Downloads\practica 3\parkinson\energy.csv
                               ch0        ch1        ch2        ch3  \
C001R_EP_reposo          10.732825  10.492954  11.380075   9.252820   
C002_EP_reposo            7.983201   8.808905  10.402469   9.827200   
C004_EP_reposo            7.074337   9.142000  14.374966   7.135363   
C005_EP_reposo_Repetido  17.655651  17.458343  19.400215  17.713516   
C006_EP_reposo            9.255415   9.869245  10.455896  10.914127   

                               ch4        ch5        ch6        ch7  
C001R_EP_reposo          14.865082  12.622079  11.390664  12.329300  
C002_EP_reposo            8.339491  46.947025  33.431248  37.842563  
C004_EP_reposo           14.393723   7.330709   7.970077   9.749949  
C005_EP_reposo_Repetido  17.952736  53.

3. Determine si existe diferencia estadística entre canales de cada grupo de sujetos

In [8]:
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
from statsmodels.stats.multitest import multipletests

channels = df_control.columns
results = []

for ch in channels:
    x = df_control[ch].values
    y = df_parkinson[ch].values

    # 1. Normalidad
    p_sh_x = shapiro(x).pvalue 
    p_sh_y = shapiro(y).pvalue 
    normal_x = (p_sh_x > 0.05)
    normal_y = (p_sh_y > 0.05)

    # 2. Homocedasticidad
    p_levene = levene(x, y).pvalue
    homo = (p_levene > 0.05)

    # 3. Selección de prueba
    if normal_x and normal_y:
        stat, pval = ttest_ind(x, y, equal_var=homo)
        test = "t-test" 
    else:
        stat, pval = mannwhitneyu(x, y)
        test = "Mann-Whitney U"

    results.append({
        "channel": ch,
        "mean_control": np.mean(x),
        "mean_parkinson": np.mean(y),
        "shapiro_control": p_sh_x,
        "shapiro_parkinson": p_sh_y,
        "levene_p": p_levene,
        "test_used": test,
        "stat": stat,
        "pval": pval
    })

res_df = pd.DataFrame(results).set_index("channel")

# Corrección por comparaciones múltiples (FDR)
#reject, pvals_corr, _, _ = multipletests(res_df["pval"], alpha=0.05, method='fdr_bh')
#res_df["pval_fdr"] = pvals_corr
#res_df["reject_fdr"] = reject

print(res_df)


         mean_control  mean_parkinson  shapiro_control  shapiro_parkinson  \
channel                                                                     
ch0         11.007337       12.174836     6.252583e-03           0.014392   
ch1         11.893459       12.690556     3.954350e-03           0.004522   
ch2         12.922938       14.732362     8.902474e-03           0.000910   
ch3         11.441595       13.003361     1.969822e-04           0.000312   
ch4         11.913825       12.894515     7.995109e-04           0.005323   
ch5         41.759198       56.293409     6.077846e-06           0.000010   
ch6         35.174209       47.129171     4.219023e-06           0.000017   
ch7         46.265497       61.770295     5.307788e-07           0.000009   

         levene_p       test_used   stat      pval  
channel                                             
ch0      0.884802  Mann-Whitney U  360.0  0.405701  
ch1      0.959093  Mann-Whitney U  377.0  0.570528  
ch2      0.977634

1. Prueba de normalidad (Shapiro-Wilk)

Todas las columnas shapiro_control y shapiro_parkinson tienen valores p < 0.05, eso significa que ningún canal sigue una distribución normal en ninguno de los dos grupos. Por eso se eligió la prueba no paramétrica Mann-Whitney U.

2. Prueba de homocedasticidad (Levene)
 Todas las columnas levene_p son p > 0.05, lo que indica igualdad de varianzas entre grupos. Aunque esto se cumple, no importa tanto aquí porque las distribuciones no eran normales.

3. Resultados de Mann-Whitney U
 Columna pval Todos los valores son mayores a 0.05 en todos los canales, no se rechaza la hipótesis nula.

 H0= No hay diferencia significativa en la energía media entre el grupo control y parkinson 
 Ha= Si hay diferencia significativa en la energía media entre el grupo control y parkinson

 4. Medias entre grupos

En todos los canales, la energía media de Parkinson es mayor que la de Control, pero esas diferencias no son estadísticamente significativas bajo las pruebas aplicadas.

Conclusión
En el análisis de energía promedio por canal, se observó que las medias de los pacientes con Parkinson fueron mayores en comparación con los controles en todos los canales. Sin embargo, las pruebas de normalidad (Shapiro-Wilk) indicaron distribuciones no normales en ambos grupos, por lo que se aplicó la prueba no paramétrica Mann-Whitney U. Ningún canal presentó diferencias significativas tras la prueba. Esto sugiere que, con los datos disponibles, la energía de los canales no discrimina entre grupos.


REFERENCIAS

Análisis de homocedasticidad y heterocedasticidad con python. (n.d.). Cienciadedatos.net. Retrieved September 22, 2025, from https://cienciadedatos.net/documentos/pystats07-test-homocedasticidad-heterocedasticidad-python

Jeong, D.-H., Kim, Y.-D., Song, I.-U., Chung, Y.-A., & Jeong, J. (2015). Wavelet energy and wavelet coherence as EEG biomarkers for the diagnosis of Parkinson’s disease-related dementia and Alzheimer’s disease. Entropy (Basel, Switzerland), 18(1), 8. https://doi.org/10.3390/e18010008

Moctezuma, L. A., & Molinas, M. (2020). EEG channel-selection method for epileptic-seizure classification based on multi-objective optimization. Frontiers in Neuroscience, 14, 593. https://doi.org/10.3389/fnins.2020.00593 
